## Data importation and preprocessing 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("satcat.tsv", delimiter="\t")

/var/folders/30/0r4tr71d3ps5xlw77dg12n180000gn/T/ipykernel_27379/3857780298.py:5: DtypeWarning: Columns (1,19,21,23,25,27,29,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("satcat.tsv", delimiter="\t")


### Time series data preprocessing

In [4]:
# Convert 'Mass' to numeric, coerce errors to NaN
df["Mass"] = pd.to_numeric(df["Mass"], errors="coerce")
df['LDate'] = pd.to_datetime(df['LDate'], errors="coerce")
df['Year'] = df['LDate'].dt.year
df = df.dropna(subset=['Year'])
df['Year'] = df['Year'].astype(int)

# satllites by year
satellites_per_year = df.groupby("Year")["Satcat"].count().reset_index(name="Satellite Count")

# Filter only debris objects (Type = 'D') and drop NaNs in Mass
df_debris = df[df["Type"].str.startswith(("D", "C"), na=False)].dropna(subset=["Mass"])

# Aggregate debris count per launch
debris_per_launch = df_debris.groupby("Launch_Tag").size().reset_index(name="Debris_Count")

# Aggregate debris count per year
debris_per_year = df_debris.groupby("Year")["Type"].count().reset_index(name="Debris Count")


### Heatmap data preprocessing

In [63]:
df_debris['Apogee'] = pd.to_numeric(df_debris['Apogee'])
df_debris['Perigee'] = pd.to_numeric(df_debris['Perigee'])

df_orbit1 = df_debris.groupby(['OpOrbit', 'Year']).size().reset_index(name='Debris Count')
df_orbit2 = df.groupby(['OpOrbit', 'Year']).size().reset_index(name='Launches Count')
df_orbit = df_orbit1.merge(df_orbit2, on=['OpOrbit', 'Year'])

# Percentage of debris in an operational orbit in each launch
df_orbit['% Debris'] = (df_orbit['Debris Count']/df_orbit['Launches Count'])* 100



### Bar graph data preprocessing

In [8]:
df['State'].unique()  #Extracting the state OrgCodes and mapping them to a country or organization name respectively

array(['SU', 'US', 'UK', 'CA', 'I', 'I-INT', 'F', 'AU', 'I-ESRO', 'D',
       'J', 'I-NATO', 'CN', 'NL', 'E', 'IN', 'I-ESA', 'ID', 'CSSR',
       'I-EUT', 'I-ARAB', 'BR', 'MX', 'S', 'I-EUM', 'IL', 'L', 'AR',
       'HKUK', 'PK', 'I-INM', 'CSFR', 'RU', 'KR', 'P', 'T', 'TR', 'CZ',
       'UA', 'MY', 'N', 'PH', 'HK', 'EG', 'CL', 'SG', 'TW', 'DK', 'ZA',
       'CYM', 'SA', 'UAE', 'MA', 'DZ', 'GR', 'NG', 'IR', 'KZ', 'CO',
       'I-RASC', 'VN', 'BM', 'VE', 'CH', 'MU', 'RO', 'HU', 'PL', 'BY',
       'KP', 'AZ', 'AT', 'EC', 'EE', 'QA', 'PE', 'BO', 'LT', 'B', 'UY',
       'I-EU', 'PG', 'MC', 'LA', 'PR', 'FI', 'IE', 'SK', 'LV', 'BG', 'GH',
       'MN', 'BD', 'AO', 'NZ', 'KE', 'CR', 'BGN', 'BT', 'JO', 'LK', 'NP',
       'SD', 'RW', 'ET', 'GT', 'SI', 'PY', 'TN', 'MYM', 'KW', 'AM', 'MD',
       'UG', 'ZW', 'DJ', 'SN', 'OM', 'HR'], dtype=object)

In [9]:
# A dictionary mapping all state OrgCodes to their respective countries/orgnanizations
orgcode_to_country = {
    "US": "United States",
    "SU": "Soviet Union",
    "RU": "Russia",
    "UK": "United Kingdom",
    "FR": "France",
    "F": "France",
    "CN": "China",
    "DE": "Germany",
    "D": "Germany (pre-unification)",
    "CA": "Canada",
    "AU": "Australia",
    "IN": "India",
    "J": "Japan",
    "KR": "South Korea",
    "IL": "Israel",
    "BR": "Brazil",
    "MX": "Mexico",
    "AR": "Argentina",
    "PK": "Pakistan",
    "SG": "Singapore",
    "TW": "Taiwan",
    "NL": "Netherlands",
    "E": "Spain",
    "IT": "Italy",
    "I": "Italy",
    "ID": "Indonesia",
    "CZ": "Czech Republic",
    "UA": "Ukraine",
    "MY": "Malaysia",
    "PH": "Philippines",
    "HK": "Hong Kong",
    "EG": "Egypt",
    "ZA": "South Africa",
    "NG": "Nigeria",
    "SA": "Saudi Arabia",
    "UAE": "United Arab Emirates",
    "IR": "Iran",
    "CO": "Colombia",
    "VE": "Venezuela",
    "CH": "Switzerland",
    "PL": "Poland",
    "RO": "Romania",
    "SE": "Sweden",
    "FI": "Finland",
    "IE": "Ireland",
    "HU": "Hungary",
    "BG": "Bulgaria",
    "GR": "Greece",
    "PT": "Portugal",
    "NZ": "New Zealand",
    "AT": "Austria",
    "BE": "Belgium",
    "SK": "Slovakia",
    "SI": "Slovenia",
    "LT": "Lithuania",
    "LV": "Latvia",
    "EE": "Estonia",
    "KZ": "Kazakhstan",
    "AZ": "Azerbaijan",
    "BY": "Belarus",
    "KP": "North Korea",
    "AM": "Armenia",
    "MD": "Moldova",
    "GH": "Ghana",
    "ET": "Ethiopia",
    "KE": "Kenya",
    "SD": "Sudan",
    "DZ": "Algeria",
    "MA": "Morocco",
    "TN": "Tunisia",
    "RW": "Rwanda",
    "UG": "Uganda",
    "ZW": "Zimbabwe",
    "PE": "Peru",
    "CL": "Chile",
    "BO": "Bolivia",
    "UY": "Uruguay",
    "EC": "Ecuador",
    "QA": "Qatar",
    "SN": "Senegal",
    "DJ": "Djibouti",
    "OM": "Oman",
    "HR": "Croatia",
    "MC": "Monaco",
    "LA": "Laos",
    "PR": "Puerto Rico",
    "BT": "Bhutan",
    "NP": "Nepal",
    "LK": "Sri Lanka",
    "BD": "Bangladesh",
    "MN": "Mongolia",
    "PG": "Papua New Guinea",
    "BGN": "Bulgaria (uncertain format)",
    "HKUK": "Hong Kong / United Kingdom",
    "T": "Thailand",
    "B": "Belgium",
    "I-ESA": "European Space Agency",
    "I-EUM": "EUMETSAT (Europe)",
    "I-NATO": "NATO",
    "I-ESRO": "ESRO (European Space Research Org)",
    "I-ARAB": "ARABSAT",
    "I-INM": "INMARSAT",
    "I-RASC": "RASC (likely Russia/Asia)",
    "I-EU": "European Union",
    "I-EUT": "EUTELSAT",
    "I-INT": "International",
}


In [100]:
df['Country/Organization'] = df['State'].map(orgcode_to_country)
df_debris['Country/Organization'] = df_debris['State'].map(orgcode_to_country)
df_debris = df_debris.dropna(subset=['Country/Organization'])

## Dashboard

In [133]:
from dash import Dash, html, dcc, Input, Output 
from dash_bootstrap_components.themes import BOOTSTRAP
import plotly.graph_objects as go
import plotly.express as px



def main():
    app = Dash(external_stylesheets=[BOOTSTRAP])
    app.title = "Satellite Dashboard"

    
    ## The layout ##
    app.layout = html.Div([
    html.H1(className="Header",
        children=["Exploring Space Debris Accumulation Caused By Satellite Deployments"],
        style={
            "textAlign": "center",
            "color": "#333",
            "marginBottom": "10px",
            "padding": "5px",
            "fontSize": "1.5 rem",
            "gridColumn": "span 2",
            "width": "100%"
        }
    ),

    # Time Series Section
    html.Div(className="TimeSeries",
             children=[
                html.H3("📈 Satellite and Debris Trend Over Time", style={"fontSize": "1.2rem", "marginBottom": "5px"}),
                dcc.Graph(id="time-series-plot", style={"height": "90%", "width": "100%"}),
                dcc.RangeSlider(
                    id='years-filter',
                    min=satellites_per_year['Year'].min(),
                    max=satellites_per_year['Year'].max(),
                    step=1,
                    value=[satellites_per_year['Year'].min(), satellites_per_year['Year'].max()],
                    marks={str(year): str(year) for year in range(satellites_per_year['Year'].min(), satellites_per_year['Year'].max(), 10)},
                    tooltip={"placement": "bottom", "always_visible": True}
                )
                
            ], style={"marginBottom": "2%", "width": "95%"}),

        
    # BarGraph Section
    html.Div(className="BarGraph",
             children=[ 
                html.H3("🌍 Country-Level Contribution to Space Debris", style={"fontSize": "1.2rem", "marginBottom": "5px"}),
                html.Div([
                    html.Div([
                        html.Label("Select Countries:", style={"fontSize": "0.9rem"}),
                        dcc.Checklist(
                            id="countries-filter",
                            options=[{'label': Country, 'value': Country} for Country in df_debris['Country/Organization'].unique()],
                            value=df_debris['Country/Organization'].unique(),
                            inputStyle={"margin-right": "6px"},
                            labelStyle={"display": "block", "margin-bottom": "2px", "fontSize": "0.85rem"},
                            style={
                                "maxHeight": "100px",  # Scroll if too many
                                "overflowY": "auto",
                                "border": "1px solid #ccc",
                                "padding": "5px",
                                "borderRadius": "5px",
                                "backgroundColor": "#f9f9f9"
                            }
                        )
                    ], style={"width": "100%"}),
                    
                    html.Div([
                        html.Label("Display Mode:", style={"fontSize": "0.9rem"}),
                        dcc.RadioItems(
                            id='display-mode',
                            options=[
                                {'label': 'Top 10 Countries', 'value': 'top'},
                                {'label': 'All Selected Countries', 'value': 'all'}
                            ],
                            value='top',
                            labelStyle={"marginRight": "5px", "fontSize": "0.85rem"}
                        )
                    ], style={"width": "35%", "display": "inline-block", "paddingLeft": "5%"})
                ], style={"marginBottom": "2%", "display": "flex", "flexDirection": "row"}),
        
                dcc.Graph(id='country-plot', style={"height": "85%", "width": "100%"})
                
            ], style={"width": "95%", "gridRow": "span 2"}),

    # Heatmap Section
    html.Div(className="HeatMap",
             children=[
                html.H3("🛰️ Debris Percentage by Orbit and Year", style={"fontSize": "1.2rem", "marginBottom": "5px"}),
                dcc.Dropdown(
                    id="OpOrbit-filter",
                    options=[{'label': OpOrbit, 'value': OpOrbit} for OpOrbit in df_orbit['OpOrbit'].unique()],
                    value=df_orbit['OpOrbit'].unique(),
                    clearable=True,
                    multi=True,
                    style={"width": "100%"}
                ),
                dcc.Graph(id="heatmap-plot", style={"height": "90%", "width": "100%"})
            ], style={"marginBottom": "2%", "width": "95%"})

    
        

    ], style={
        "display": "grid",
        "gridTemplateColumns": "2fr 1fr",
        "gridTemplateAreas": 
        """
            'Header Header'
            'TimeSeries BarGraph'
            'HeatMap BarGraph'
        """,  
        "width": "100%",
        "maxWidth": "1200px",
        "margin": "10", 
        "minHeight": "100vh",
        "alignItems": "start",
        "padding": "10px",
        "gap": "10px",
        "boxSizing": "border-box",
        "justifyItems": "center",
    })

    
    ## Callbacks ##


    @app.callback(
        Output("time-series-plot", 'figure'),
        Input("years-filter", "value")
    )
    def update_time_series(selected_years):
        if not isinstance(selected_years, list):
            selected_years = [selected_years]
            
        start_year, end_year = selected_years

        satellites = satellites_per_year[
            (satellites_per_year['Year'] >= start_year) & 
            (satellites_per_year['Year'] <= end_year)
        ]
        
        debris = debris_per_year[
            (debris_per_year['Year'] >= start_year) & 
            (debris_per_year['Year'] <= end_year)
        ]
        
        
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=satellites['Year'],
                y=satellites['Satellite Count'],
                mode='lines+markers',
                name='Satellites'
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=debris["Year"],
                y=debris["Debris Count"],
                mode='lines+markers',
                name='Debris'
            )
        )
        
        fig.update_layout(
             title={
            "text": "Satellite Launches Over Time (1957 - 2025)",
            "x": 0.5,
            "xanchor": "center",
            "font": {
                "size": 18
                }
             },
            
            xaxis=dict(
                title="Year",
                tickmode='linear',
                dtick=5 
                ),
            
            yaxis_title="Number of Satellites/Debris",
            title_font_size=18,
            width=900,
            height=400
        )

        return fig

    @app.callback(
        Output("heatmap-plot", "figure"),
        Input("OpOrbit-filter", "value"),
        Input("years-filter", "value")
    )
    def update_heatmap(selected_orbits, selected_years):
        if not isinstance(selected_years, list):
                selected_years = [selected_years]
        if not isinstance(selected_orbits, list):
                selected_orbits = [selected_orbits]
        start_year, end_year = selected_years
       
        
        # Pivot to create a 2D matrix: rows = OpOrbit, columns = Year, values = % Debris
        data = df_orbit[(df_orbit['OpOrbit'].isin(selected_orbits)) & ((df_orbit['Year'] >= start_year) & (df_orbit['Year'] <= end_year))]
        heatmap_data = data.pivot(index="OpOrbit", columns="Year", values="% Debris").fillna(0)
        
        
        # Plot
        fig = px.imshow(
            heatmap_data,
            labels=dict(x="Year", y="Operational Orbit", color="% Debris"),
            color_continuous_scale="Reds",
            aspect="auto"
        )
        
        # Optional: Make x-axis categorical to preserve order
        fig.update_xaxes(type='category')
        fig.update_layout(
        
            title={
                    "text": "% Debris by Operational Orbit and Year",
                    "x": 0.5,
                    "xanchor": "center",
                    "font": {
                        "size": 18
                        }
                     },
            width=900,
            height=400
        )
        
        return fig


    @app.callback(
        Output("country-plot", "figure"),
        Input("years-filter", "value"),
        Input("countries-filter", "value"),
        Input("display-mode", "value")
    )
    def update_bargraph_plot(selected_years, selected_countries, display_mode):
        if not isinstance(selected_years, list):
            selected_years = [selected_yars]
        if not isinstance(selected_countries, list):
            selected_countries = [selected_countries]
        start_year, end_year = selected_years
        filtered_df = df_debris[((df_debris['Year'] >= start_year) & (df_debris['Year'] <= end_year)) & (df_debris['Country/Organization'].isin(selected_countries))]
    
        top_countries = filtered_df['Country/Organization'].value_counts().reset_index()
        top_countries.columns = ['Country/Organization', 'Debris Count']
        top_countries['% Debris'] = (top_countries['Debris Count']/top_countries['Debris Count'].sum())*100

        if display_mode == 'top':
            top_countries = top_countries.head(10)
    
        fig = px.bar(
            top_countries.sort_values("% Debris"),
            x="% Debris",
            y="Country/Organization",
            orientation='h',
            color="% Debris",
            color_continuous_scale='Reds'
        )
        fig.update_layout(
        
            title={
                    "text": "Top Countries Contributing to Space Debris",
                    "x": 0.5,
                    "xanchor": "center",
                    "font": {
                        "size": 18
                        }
                     },
            width=700,
            height=800
        )
        
        return fig


        
    import webbrowser
    from threading import Timer
    
    port = 8050  # or any free port
    
    def open_browser():
        webbrowser.open_new(f"http://localhost:{port}/")
    
    # Start the browser *after* the server has started
    Timer(1, open_browser).start()
    
    app.run(debug=True, use_reloader=False, port=port)


if __name__ == "__main__":
    main()


    
    